In [4]:
import pandas as pd
class DataHandler:
    def __init__(self):
        self.df_prices = None
        self.df_listing = None
        self.df_merged = None
    def get_data(self):
        self.df_prices = pd.read_csv("datas/price_availability.csv", sep=";")
        self.df_listing = pd.read_csv("datas/listings_final.csv", sep=";")
    def group_data(self):
        df_merged = pd.merge(self.df_listing, self.df_prices, on='listing_id')
        mean_prices = df_merged.groupby(['listing_id'])["local_price"].mean()
        self.df_prices = pd.merge(self.df_prices, mean_prices, on='listing_id')
        self.df_merged = pd.merge(self.df_listing, self.df_prices, on='listing_id')
        print(self.df_merged)
    def get_process_data(self):
        self.get_data()
        self.group_data()

In [5]:
data = DataHandler()
data.get_process_data()

        Unnamed: 0  listing_id              name          type   city  \
0                0    28581061   La maison Clery  private_room  Paris   
1                0    28581061   La maison Clery  private_room  Paris   
2                0    28581061   La maison Clery  private_room  Paris   
3                0    28581061   La maison Clery  private_room  Paris   
4                0    28581061   La maison Clery  private_room  Paris   
...            ...         ...               ...           ...    ...   
402611         999    28682896  Village Jourdain  private_room  Paris   
402612         999    28682896  Village Jourdain  private_room  Paris   
402613         999    28682896  Village Jourdain  private_room  Paris   
402614         999    28682896  Village Jourdain  private_room  Paris   
402615         999    28682896  Village Jourdain  private_room  Paris   

                        neighborhood   latitude  longitude  person_capacity  \
0                  2e arrondissement  48.869